In [76]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.
school_data_complete = pd.merge(student_data, school_data, how="left", on="school_name")
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary


In [77]:
school_count = school_data['school_name'].nunique()
school_count

15

In [78]:
# Calculate the total number of students
student_count = student_data['student_name'].nunique()
student_count

32715

In [79]:
# Calculate the total budget
total_budget = school_data['budget'].sum()
total_budget


24649428

In [80]:
# Calculate the average math score
average_math_score = student_data['math_score'].mean()
average_math_score

78.98537145774827

In [81]:
# Calculate the average reading score
average_reading_score = student_data['reading_score'].mean()
average_reading_score

81.87784018381414

In [82]:
passing_math_count = (school_data_complete['math_score'] >= 70).sum() / len(school_data_complete) * 100
passing_math_count

74.9808526933878

In [83]:
passing_reading_count = (school_data_complete['reading_score'] >= 70).sum() / len(school_data_complete) * 100
passing_reading_count


85.80546336482001

In [84]:
overall_passing = ((school_data_complete['math_score'] >= 70) & (school_data_complete['reading_score'] >= 70)).sum() / len(school_data_complete) * 100
overall_passing

65.17232575950983

In [85]:
district_summary = pd.DataFrame({
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Math Score": [average_math_score],
    "Average Reading Score": [average_reading_score],
    "% Passing Math": [passing_math_count],
    "% Passing Reading": [passing_reading_count],
    "% Overall Passing": [overall_passing]
})

district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.2f}%".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.2f}%".format)
district_summary["% Overall Passing"] = district_summary["% Overall Passing"].map("{:.2f}%".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"32,715","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,65.17%


## School Summary

In [86]:
merged_data = pd.merge(student_data, school_data, how="left", on="school_name")

# Group data by school name and calculate metrics
school_summary = merged_data.groupby("school_name").agg(
    School_Type=("type", "first"),
    Total_Students=("Student ID", "nunique"),
    Total_School_Budget=("budget", "first"),
    Average_Math_Score=("math_score", "mean"),
    Average_Reading_Score=("reading_score", "mean")
)

In [96]:
school_types = school_data.set_index(["school_name"])["type"]
school_types

school_name
Huang High School        District
Figueroa High School     District
Shelton High School       Charter
Hernandez High School    District
Griffin High School       Charter
Wilson High School        Charter
Cabrera High School       Charter
Bailey High School       District
Holden High School        Charter
Pena High School          Charter
Wright High School        Charter
Rodriguez High School    District
Johnson High School      District
Ford High School         District
Thomas High School        Charter
Name: type, dtype: object

In [97]:
per_school_counts =school_data[['school_name', 'size']]
per_school_counts


,school_name,size
0,Huang High School,2917
1,Figueroa High School,2949
2,Shelton High School,1761
3,Hernandez High School,4635
4,Griffin High School,1468
5,Wilson High School,2283
6,Cabrera High School,1858
7,Bailey High School,4976
8,Holden High School,427
9,Pena High School,962


In [99]:
per_school_budget = school_data[['school_name', 'budget']].rename(
    columns={'school_name': 'School Name', 'budget': 'Total School Budget'}
)

per_school_capita =school_data['Per Student Budget'] = school_data['budget'] / school_data['size']
per_school_capita

0     655.0
1     639.0
2     600.0
3     652.0
4     625.0
5     578.0
6     582.0
7     628.0
8     581.0
9     609.0
10    583.0
11    637.0
12    650.0
13    644.0
14    638.0
dtype: float64

In [90]:
per_school_math = average_math_score = student_data.groupby('school_name')['math_score'].mean().reset_index()

per_school_reading = student_data.groupby('school_name')['reading_score'].mean().reset_index().rename(
    columns={'school_name': 'School Name', 'reading_score': 'Average Reading Score'}
)

In [91]:
students_passing_math = student_data[student_data['math_score'] >= 70].groupby('school_name')['Student ID'].count()
total_students = student_data.groupby('school_name')['Student ID'].count()

school_students_passing_math =(students_passing_math / total_students * 100).reset_index().rename(
    columns={'school_name': 'School Name', 'Student ID': '% Passing Math'}
)

In [92]:
students_passing_reading = student_data[student_data['reading_score'] >= 70].groupby('school_name')['Student ID'].count()
total_students = student_data.groupby('school_name')['Student ID'].count()

school_students_passing_reading = (students_passing_reading / total_students * 100).reset_index().rename(
    columns={'school_name': 'School Name', 'Student ID': '% Passing Reading'}
)


In [93]:
students_passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()

In [94]:
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100


In [ ]:
per_school_summary = pd.DataFrame({
    "School Type" : [school_types],
    "Student counts" : [per_school_counts],
    "Total School Budget" : [per_school_budget],
    "Per Student Budget" : [per_school_capita],
    "Average Math Score" : [per_school_math],
    "Average Reading Score": [per_school_reading],
    "% Passing Math": [school_students_passing_math],
    "% Passing Reading": [school_students_passing_math],
    "% Overall Passing": [school_students_passing_math_and_reading]
})

per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)
per_school_summary["% Passing Math"] = per_school_summary["% Passing Math"].map("{:.2f}%".format)
per_school_summary["% Passing Reading"] = per_school_summary["% Passing Reading"].map("{:.2f}%".format)
per_school_summary["% Overall Passing"] = per_school_summary["% Overall Passing"].map("{:.2f}%".format)

per_school_summary

ValueError: Cannot specify ',' with 's'.

## Highest Performing Schools (by % Overall Passing)

In [109]:
top_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=False).head(5)
top_schools

,School Type,Student counts,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,school_name Huang High School District ...,school_name size 0 Huang ...,School Name Total School Budget...,0 655.0 1 639.0 2 600.0 3 652....,school_name math_score 0 B...,School Name Average Reading Sco...,School Name % Passing Math 0 ...,School Name % Passing Math 0 ...,school_name Bailey High School 2719 Cabr...
